In [9]:
using JuMP, Gurobi, CSV, DataFrames, Statistics, Plots, Random
#LinearAlgebra, StatsBase, StatsBase, DelimitedFiles
# genv = Gurobi.Env();

┌ Info: Recompiling stale cache file C:\Users\Danying Xiao\.julia\compiled\v1.2\Plots\ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1240


In [2]:
test = CSV.read("testv1.csv")
test
by(test, :stay_date, nrow)

,stay_date,x1
,Dates…,Int64
1,2014-04-10,17
2,2014-04-11,17
3,2014-04-12,43
4,2014-04-13,43
5,2013-06-30,919
6,2013-07-01,702
7,2013-09-04,198
8,2013-09-05,100
9,2013-09-06,66


#### Formulation:

**Variables:**

$m = 38: $ Number of Search Ranks that can be displayed at a Page

$n = 100: $ Number of listing-nights

$R_k: $ Rank Multiplier, $\forall k = 1,...,m$

$\alpha: \% $ of Booking Revenue Expedia receives per Booking

$W_a: $ Day of Week Multiplier, $\forall a = 1,...,7$

$M_b: $ Month Multiplier, $\forall b = 1,...,12$

$P_i: $ Price at Listing-night i, $\forall i = 1,...,n$


$
\begin{align}
 Z_{i,j} = \begin{cases}
    1 &\text{if listing-night i in hotel group j} \\
0 &\text{otherwise}
\end{cases} \\
\end{align}
$

$\tau(IV_i) $ is the inverse of the demand function which gives the booking probability

$
\begin{align}
 \tau_c(IV_i) = \begin{cases}
    \tau_{True} (IV_i) = \frac{log(\frac{IV_i}{a_{True}})}{b_{True}}&\text{where $a_{True}$ and $b_{True}$ are the coefficients of the true demand curve} \\
\tau_{OCT} (IV_i) = \frac{log(\frac{IV_i}{a_{OCT}})}{b_{OCT}} &\text{where $a_{OCT}$ and $b_{OCT}$ are the coefficients of the OCT predicted demand curve }
\end{cases} \\
\end{align}
$

Decision Variables:

$
\begin{align}
 X_{i,k} = \begin{cases}
    1 &\text{if listing-night i is assigned to rank k} \\
0 &\text{otherwise}
\end{cases} \\
\end{align}
$

$$ 
\begin{align}
\underset{X_{i,k}}{max\ } & \alpha \sum_i^n \left[P_i\times\tau_c(IV_i)\times\sum_k^n (R_kX_{i,k})  \right]  \\
\text{s.t. } & IV_i = \frac{P_i}{W_aM_b} \forall i = 1,...,n \ \ \ &\text{(i)}\\
             & \sum_i^n X_{i,k} = 1, \forall k =1,...,m \ \ \ &\text{(ii)}\\
             & \sum_k^n X_{i,k} <= 1, \forall i =1,...,n\ \ \ &\text{(iii)}\\
             & \sum_i^n\sum_k^{10} X_{i,k}Z_{i,j} >=1, \forall j=1,...,p \ \ \ &\text{(iv)}\\
             & \text{where p is the number of hotel clusters}
\end{align}
$$

In [3]:
#Load in data
rank_mult = CSV.read("rank_multv1.csv", header=1); #Vector of search rank multipliers of p by 1
rank_mult = rank_mult[!,2]

month_mult = CSV.read("month_multv1.csv", header=1); #Vector of monthly multipliers of 12 by 1
month_mult = month_mult[!,2]

week_mult = CSV.read("week_multv1.csv", header=1); #Vector of day of week multipliers of 7 by 1
week_mult = week_mult[!,2]

7-element CSV.Column{Float64,Float64}:
 1.0               
 0.9707128970695779
 0.9671885422797728
 1.052249042374722 
 1.2525982712242012
 1.296769824495693 
 1.1086168550505864

In [4]:
#function to get intrinsic px
function intrinsicF(listing_px_i,mth_mult,week_mult)
    return listing_px * mth_mult * week_mult
end

#demand function
function demandF(iv,pred_true,group)
    if pred_true == 1
        if group == 0
            a = 305.27417277; b= -12.42761648;
        elseif group == 1
            a = 253.54593267; b= -37.53117532;
        elseif group == 2
            a = 185.43176503; b= -29.15210551;
        elseif group == 3
            a = 182.44087022; b= -33.85116882;
        else
            a = 190.07289677; b= -39.04852811;
        end
    else
        if group == 0
            a = 233.41661873; b= -41.25055808;
        elseif group == 1
            a = 379.40583405; b= -37.19840181;
        elseif group == 2
            a = 415.46424746; b= -66.78216454;
        elseif group == 3
            a = 301.73547847; b= -27.51108341;
        else
            a = 161.44049996; b= -12.48533166;
        end
    end
    booking_prob = log(iv/a)/b
    return booking_prob
end

demandF (generic function with 1 method)

In [10]:
LP_0 = test[test.Hotel_Group .== 0,:].price_usd;
LP_1 = test[test.Hotel_Group .== 1,:].price_usd;
LP_2 = test[test.Hotel_Group .== 2,:].price_usd;
LP_3 = test[test.Hotel_Group .== 3,:].price_usd;
LP_4 = test[test.Hotel_Group .== 4,:].price_usd;
shuffle!(LP_0); shuffle!(LP_1); shuffle!(LP_2); shuffle!(LP_3); shuffle!(LP_4);
sample = vcat(LP_0[1:10], LP_1[1:10], LP_2[1:10], LP_3[1:10],LP_4[1:10]) #Sample Listing Prices

50-element Array{Float64,1}:
  92.0 
 119.0 
  83.0 
  49.0 
  59.0 
 151.0 
 136.0 
  33.0 
  52.0 
  55.0 
 249.0 
  98.04
 320.96
   ⋮   
  86.0 
 259.0 
  80.36
 160.41
 262.17
  87.23
  33.04
 199.17
 135.01
  85.55
 165.75
 245.96

In [6]:
n = 50
M1 = zeros(n,5)
[M1[i,1] = 1 for i=1:10]; [M1[i,2] = 1 for i=11:20]; [M1[i,3] = 1 for i=21:30]; [M1[i,4] = 1 for i=31:40]; 
[M1[i,5] = 1 for i=41:50];

M2 = 5 # May
M3 = 6 # Sunday

6

In [24]:
#Given listing prices of 100 hotels, rank the hotels so as to maximize total expected revenue for Expedia
#Sets: Hotels(listing-night) h C H, for h=1:m. Clusters c C, for c=1:n, Rank r C R, for r=1:p
#Params: Demand f(x) D_c(IV_h) from OCT and True, Hotel rank HR_h, 

m = 50; #100 listing-nights
n = 5; #5 clusters
p = 38; #38 search results shown
s = 0.05; #% of total booking revenue paid to Expedia for each hotel
P = sample; #Listing Price of listing-night i. Vector of m by 1
M1; #Matrix of m by n. 1 if hotel i belongs to cluster j, 0 o/w
M2; #Month of listing-night i
M3; #Day_of_week of listing-night i

month_mult; #Vector of monthly multipliers. 12 by 1
week_mult; #Vector of day of week multipliers. 7 by 1
rank_mult; #Vector of search rank multipliers of p by 1

mio = Model(solver=GurobiSolver(OutputFlag = 0))
    @variable(mio, x[1:m, 1:p], Bin) #Indicator if hotel i is shown in position p
    @objective(mio, Max, s * sum(P[i] * demandF(P[i], 0, findall(x->x>0,M1[i,:])) * sum(x[i,k] * rank_mult[k] for k=1:p) for i=1:38)) #Max total rank-weighted revenue for first 38 hotels
        
    @constraint(mio, [k=1:p], sum(x[i,k] for i=1:m) == 1) #1 hotel per search rank    
    @constraint(mio, [i=1:m], sum(x[i,k] for k=1:p) <= 1) #a hotel can only be  rank at most once
    @constraint(mio, [j=1:n], sum(x[i,k] * M1[i,j] for k=1:10, i=1:m) >= 1 ) #Diversity: at least 1 hotel from each cluster in top 10 results
    solve(mio)
    rankmatrix = getvalue(x)
    revenue = getobjectivevalue(mio)
    return(getvalue(x))

Academic license - for non-commercial use only


50×38 Array{Float64,2}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0   0.0   1.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0   1.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   0.0   1.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   0.0   1.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 

In [25]:
revenue

7.349083381289892